# Model Context Protocol (MCP) Demo with Anthropic Messages API

This notebook demonstrates the basic usage of Anthropic's Model Context Protocol (MCP) for connecting AI models to external data sources and tools.

## What is MCP?

MCP (Model Context Protocol) is an open standard by Anthropic that standardizes how AI models connect to external tools and data sources. Think of it as a "USB-C port for AI applications" - it provides a unified way to connect to different services instead of building custom integrations for each one.

## Key Benefits:
- **Standardized**: One protocol for all external connections
- **Secure**: Built-in authentication and security practices
- **Flexible**: Works with local and remote servers
- **Open**: Not limited to Anthropic - adopted by OpenAI and others

## Required Environment Variables

```
ANTHROPIC_API_KEY    # Your Anthropic API key (required)
```

Please refer to the [README](README.md) for instructions on setting up environment variables.

## Setup

First, let's install the required packages and set up our environment.

In [ ]:
# If running standalone without the project setup:
# pip install anthropic python-dotenv requests mcp nest-asyncio

In [ ]:
import os
import json
import requests
import asyncio
from anthropic import Anthropic
from dotenv import load_dotenv
from mcp.server.fastmcp import FastMCP
import nest_asyncio

# Enable nested asyncio for Jupyter
nest_asyncio.apply()

# Load environment variables
load_dotenv(override=True)

# Initialize Anthropic client
client = Anthropic(
    api_key=os.getenv('ANTHROPIC_API_KEY')  # Make sure to set your API key in .env file
)

## Basic MCP Configuration

Let's set up a basic MCP server configuration. For this demo, we'll create a mock MCP server endpoint that simulates a weather service.

In [ ]:
# MCP Server configuration
# In a real scenario, this would be a running MCP server
# For demo purposes, we'll simulate the MCP interaction

mcp_server_config = {
    "type": "url",
    "url": "https://api.openweathermap.org/data/2.5",  # Using OpenWeatherMap as example
    "name": "weather-api",
    "authorization_token": "demo_token",  # In real usage, get proper API key
    "tool_configuration": {
        "enabled": True,
        "allowed_tools": ["get_weather"]
    }
}

print("MCP Server Configuration:")
print(json.dumps(mcp_server_config, indent=2))

## MCP Demo with External API

Since setting up a full MCP server requires additional infrastructure, we'll demonstrate the MCP concept using Anthropic's Messages API with the MCP connector feature.

**Important:** The MCP connector feature requires the beta header `"anthropic-beta": "mcp-client-2025-04-04"`

## Real MCP Server Implementation

Let's create an actual MCP server for weather data using the Open-Meteo API (free, no API key required). This will demonstrate how MCP servers expose tools that AI models can use.

In [ ]:
# Create a simple MCP server for weather data
# Using Open-Meteo API (free, no API key required)

mcp = FastMCP("weather-server")

@mcp.tool()
async def get_weather(location: str = "Singapore") -> dict:
    """
    Get current weather for a location.
    Defaults to Singapore if no location specified.

    Args:
        location: City name (e.g., "Singapore", "Tokyo", "London")

    Returns:
        Current weather data including temperature, conditions, and more
    """
    # Coordinates for common cities (for demo simplicity)
    city_coords = {
        "singapore": {"lat": 1.3521, "lon": 103.8198},
        "tokyo": {"lat": 35.6762, "lon": 139.6503},
        "london": {"lat": 51.5074, "lon": -0.1278},
        "new york": {"lat": 40.7128, "lon": -74.0060},
        "sydney": {"lat": -33.8688, "lon": 151.2093}
    }

    # Get coordinates for the city
    coords = city_coords.get(location.lower(), city_coords["singapore"])

    # Fetch weather from Open-Meteo API (free, no key required)
    url = f"https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": coords["lat"],
        "longitude": coords["lon"],
        "current": "temperature_2m,relative_humidity_2m,weather_code,wind_speed_10m",
        "timezone": "auto"
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()

        # Weather code descriptions
        weather_codes = {
            0: "Clear sky",
            1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
            45: "Foggy", 48: "Depositing rime fog",
            51: "Light drizzle", 53: "Moderate drizzle", 55: "Dense drizzle",
            61: "Slight rain", 63: "Moderate rain", 65: "Heavy rain",
            71: "Slight snow", 73: "Moderate snow", 75: "Heavy snow",
            77: "Snow grains",
            80: "Slight rain showers", 81: "Moderate rain showers", 82: "Violent rain showers",
            85: "Slight snow showers", 86: "Heavy snow showers",
            95: "Thunderstorm", 96: "Thunderstorm with slight hail", 99: "Thunderstorm with heavy hail"
        }

        current = data["current"]
        weather_code = current.get("weather_code", 0)

        return {
            "location": location.title(),
            "temperature": f"{current['temperature_2m']}°C",
            "condition": weather_codes.get(weather_code, "Unknown"),
            "humidity": f"{current['relative_humidity_2m']}%",
            "wind_speed": f"{current['wind_speed_10m']} km/h",
            "source": "Open-Meteo API via MCP",
            "timestamp": current["time"]
        }
    except Exception as e:
        return {"error": f"Failed to fetch weather: {str(e)}"}

## Testing the MCP Weather Tool

Let's test our MCP weather tool by fetching real weather data for Singapore:

In [ ]:
# Test the MCP tool directly - fetch real Singapore weather
print("🌤️ Fetching real weather data for Singapore via MCP...\n")
singapore_weather = asyncio.run(get_weather("Singapore"))
print("Singapore Weather (Real-time data):")
print(json.dumps(singapore_weather, indent=2))

print("\n" + "="*50 + "\n")

# Also test for another city
print("🌍 Fetching weather for Tokyo...\n")
tokyo_weather = asyncio.run(get_weather("Tokyo"))
print("Tokyo Weather (Real-time data):")
print(json.dumps(tokyo_weather, indent=2))

## Using Anthropic Messages API with MCP Concept

Now let's demonstrate how this would work with Anthropic's Messages API. We'll create a conversation where Claude needs to fetch external data to answer a question.

In [ ]:
# Define tools for Claude to use (these match our MCP server tools)
tools = [
    {
        "name": "get_weather",
        "description": "Get current weather information for a location. Defaults to Singapore if no location specified.",
        "input_schema": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city to get weather for (e.g., Singapore, Tokyo, London)"
                }
            },
            "required": []
        }
    }
]

print("Available MCP tools:")
for tool in tools:
    print(f"- {tool['name']}: {tool['description']}")

In [ ]:
# Tool execution function (connecting to our MCP server)
def execute_tool(tool_name, tool_input):
    """
    Execute a tool call - this connects to our MCP server
    """
    if tool_name == "get_weather":
        # Call our actual MCP weather tool
        location = tool_input.get("location", "Singapore")
        result = asyncio.run(get_weather(location))
        return result
    else:
        return {"error": f"Unknown tool: {tool_name}"}

# Test tool execution with real data
print("🌤️ Testing MCP tool execution for Singapore:")
test_singapore = execute_tool("get_weather", {"location": "Singapore"})
print(json.dumps(test_singapore, indent=2))

print("\n" + "="*50 + "\n")

print("🌏 Testing MCP tool execution for Tokyo:")
test_tokyo = execute_tool("get_weather", {"location": "Tokyo"})
print(json.dumps(test_tokyo, indent=2))

## Complete MCP-Style Conversation Demo

Let's create a complete example showing how Claude would use MCP-style tools to answer user questions.

In [ ]:
# Simulate a conversation with tool usage
def run_mcp_conversation(user_message):
    print(f"User: {user_message}\n")

    # First API call to Claude
    response = client.messages.create(
        model="claude-3-5-haiku-20241022",
        max_tokens=1000,
        tools=tools,
        messages=[
            {
                "role": "user",
                "content": user_message
            }
        ]
    )

    print("Claude's response:")

    # Check if Claude wants to use tools
    if response.stop_reason == "tool_use":
        print("Claude is calling MCP tools to get real weather data...\n")

        # Extract tool calls
        tool_results = []
        messages = [
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": response.content}
        ]

        for content_block in response.content:
            if content_block.type == "tool_use":
                tool_name = content_block.name
                tool_input = content_block.input
                tool_id = content_block.id

                print(f"🔧 Using MCP tool: {tool_name}")
                print(f"📥 Input: {tool_input}")

                # Execute the tool (real MCP call)
                tool_result = execute_tool(tool_name, tool_input)

                print(f"📤 Real data from MCP:")
                print(json.dumps(tool_result, indent=4))
                print()

                # Add tool result to conversation
                tool_results.append({
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_id,
                            "content": json.dumps(tool_result)
                        }
                    ]
                })

        # Add tool results to messages
        messages.extend(tool_results)

        # Get final response from Claude
        final_response = client.messages.create(
            model="claude-3-5-haiku-20241022",
            max_tokens=1000,
            tools=tools,
            messages=messages
        )

        print("Claude's final answer (based on real data):")
        for content_block in final_response.content:
            if content_block.type == "text":
                print(content_block.text)

    else:
        # Claude answered without tools
        for content_block in response.content:
            if content_block.type == "text":
                print(content_block.text)

    print("\n" + "="*50 + "\n")

## MCP-Style Conversation Demo with Real Weather API

Now let's see how Claude uses our MCP weather tool to answer questions with real-time data:

In [ ]:
# Demo 1: Singapore weather query (real-time data!)
print("🇸🇬 SINGAPORE WEATHER DEMO\n")
print("This will fetch REAL weather data for Singapore using MCP...\n")
run_mcp_conversation("What's the current weather in Singapore? Please provide all details.")

In [ ]:
# Demo 2: Weather comparison
print("🌏 WEATHER COMPARISON DEMO\n")
run_mcp_conversation("Can you compare the current weather between Singapore and Tokyo? Which city is warmer?")

In [ ]:
# Demo 3: Travel planning with real weather data
print("✈️ TRAVEL PLANNING DEMO\n")
run_mcp_conversation("I'm planning to travel from Singapore to London next week. How's the weather difference between these two cities right now?")